<a href="https://colab.research.google.com/github/Zalunda/FFmpeg/blob/master/TestCollabEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# Check GPU presence
!nvidia-smi

# Update apt and install CUDA toolkit (if not fully present)
# Colab usually has CUDA pre-installed, but this ensures dev tools are there.
!apt update -y
!apt install -y nvidia-cuda-toolkit

# Verify CUDA installation path (should be /usr/local/cuda)
!ls -l /usr/local/cuda

Mon May 26 14:20:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [45]:
# Install essential build tools and libraries
!apt install -y build-essential yasm nasm cmake libtool unzip wget libnuma-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libnuma-dev is already the newest version (2.0.14-3ubuntu2).
libtool is already the newest version (2.4.6-15build2).
nasm is already the newest version (2.15.05-1).
yasm is already the newest version (1.3.0-2.1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.


In [46]:
# Create a working directory
!mkdir -p ~/nvidia
%cd ~/nvidia/

# Clone NVIDIA Codec Headers
!rm -rf nv-codec-headers
!git clone https://git.videolan.org/git/ffmpeg/nv-codec-headers.git

%cd nv-codec-headers
!make install # This installs headers to /usr/local/include

# Verify what's in the ~/nvidia/ directory from the current location (nv-codec-headers)
# Note: At this point, the 'ffmpeg' directory likely won't be there yet,
# as it's cloned in a later step.
%ls ~/nvidia/

# --- FIX FOR ERROR 2: Correct comment syntax for %cd ---
# Go back to the main nvidia directory
%cd ~/nvidia/

# Verify contents of the current directory (should be ~/nvidia/)
%ls

/root/nvidia
Cloning into 'nv-codec-headers'...
remote: Enumerating objects: 1075, done.
remote: Counting objects: 100% (1075/1075), done.
remote: Compressing objects: 100% (913/913), done.
remote: Total 1075 (delta 576), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (1075/1075), 194.70 KiB | 486.00 KiB/s, done.
Resolving deltas: 100% (576/576), done.
/root/nvidia/nv-codec-headers
sed 's#@@PREFIX@@#/usr/local#' ffnvcodec.pc.in > ffnvcodec.pc
install -m 0755 -d '/usr/local/include/ffnvcodec'
install -m 0644 include/ffnvcodec/*.h '/usr/local/include/ffnvcodec'
install -m 0755 -d '/usr/local/lib/pkgconfig'
install -m 0644 ffnvcodec.pc '/usr/local/lib/pkgconfig'
ffmpeg/  nv-codec-headers/
/root/nvidia
ffmpeg/  nv-codec-headers/


In [47]:
# Clone FFmpeg source code
!rm -rf ffmpeg
#!git clone https://github.com/Zalunda/FFmpeg.git ffmpeg/
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg/

Cloning into 'ffmpeg'...
remote: Enumerating objects: 15657, done.
remote: Counting objects: 100% (15657/15657), done.
remote: Compressing objects: 100% (8346/8346), done.
remote: Total 760885 (delta 11136), reused 9933 (delta 7293)
Receiving objects: 100% (760885/760885), 188.73 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (616034/616034), done.


In [48]:
%cd ~/nvidia/ffmpeg/

# Configure FFmpeg with NVIDIA support
# --enable-nonfree: Required for NVENC/NVDEC (they are proprietary)
# --enable-cuda-nvcc: Enables compilation with NVCC
# --enable-libnpp: Enables NVIDIA Performance Primitives (often used for GPU filters)
# --extra-cflags/ldflags: Ensure compiler finds CUDA headers and libraries
!./configure \
    --enable-nonfree \
    --enable-cuda-nvcc \
    --enable-libnpp \
    --extra-cflags=-I/usr/local/cuda/include \
    --extra-ldflags=-L/usr/local/cuda/lib64 \
    --disable-ffplay \
    --disable-ffprobe

# Optional: Add other common libraries if you need them (uncomment and install via apt first)
# --enable-libx264 \
# --enable-libx265 \
# --enable-libvpx \
# --enable-libopus \
# --enable-libmp3lame \
# --enable-gpl # Required for x264, x265 etc.

/root/nvidia/ffmpeg
install prefix            /usr/local
source path               .
C compiler                gcc
C library                 glibc
ARCH                      x86 (generic)
big-endian                no
runtime cpu detection     yes
standalone assembly       yes
x86 assembler             nasm
MMX enabled               yes
MMXEXT enabled            yes
3DNow! enabled            yes
3DNow! extended enabled   yes
SSE enabled               yes
SSSE3 enabled             yes
AESNI enabled             yes
AVX enabled               yes
AVX2 enabled              yes
AVX-512 enabled           yes
AVX-512ICL enabled        yes
XOP enabled               yes
FMA3 enabled              yes
FMA4 enabled              yes
i686 features enabled     yes
CMOV is fast              yes
EBX available             yes
EBP available             yes
debug symbols             yes
strip symbols             yes
optimize for size         no
optimizations             yes
static                    yes
shar

In [ ]:
# Compile FFmpeg
!make -j $(nproc)

GEN	libavutil/libavutil.version
GEN	libswscale/libswscale.version
GEN	libswresample/libswresample.version
GEN	libavcodec/libavcodec.version
GEN	libavformat/libavformat.version
GEN	libavfilter/libavfilter.version
GEN	libavdevice/libavdevice.version
CC	libavdevice/alldevices.o
CC	libavdevice/alsa.o
CC	libavdevice/alsa_dec.o
CC	libavdevice/alsa_enc.o
CC	libavdevice/avdevice.o
CC	libavdevice/fbdev_common.o
CC	libavdevice/fbdev_dec.o
CC	libavdevice/fbdev_enc.o
CC	libavdevice/lavfi.o
CC	libavdevice/oss.o
CC	libavdevice/oss_dec.o
CC	libavdevice/oss_enc.o
CC	libavdevice/timefilter.o
CC	libavdevice/utils.o
CC	libavdevice/v4l2-common.o
CC	libavdevice/v4l2.o
CC	libavdevice/v4l2enc.o
CC	libavdevice/version.o
CC	libavdevice/xcbgrab.o
GEN	libavdevice/libavdevice.pc
CC	libavfilter/aeval.o
CC	libavfilter/af_aap.o
CC	libavfilter/af_acontrast.o
CC	libavfilter/af_acopy.o
CC	libavfilter/af_acrossover.o
CC	libavfilter/af_acrusher.o
CC	libavfilter/af_adeclick.o
CC	libavfilter/af_adecorrelate.o
CC	libavfilte

In [ ]:
# Check if the ffmpeg executable was created
!ls -l ffmpeg

# Run a basic version check
!./ffmpeg -version

# Verify NVENC and NVDEC are available
!./ffmpeg -encoders | grep nvenc
!./ffmpeg -decoders | grep h264_cuvid # h264_cuvid or h264_nvdec are common NVIDIA decoders

# Install FFmpeg to /usr/local/bin so it's globally accessible
!sudo make install

# Verify global installation
!ffmpeg -version